PWC-Net-small model finetuning (with cyclical learning rate schedule)
=======================================================

In this notebook we:
- Use a small model (no dense or residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the PWC-Net-small model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a Cyclic<sub>short</sub> schedule of our own
- Let the Cyclic<sub>short</sub> schedule oscillate between `2e-05` and `1e-06` for 200,000 steps
- Switch to the "robust" loss described in the paper, instead of the "multiscale" loss used during training

Below, look for `TODO` references and customize this notebook based on your own needs.

## Reference

[2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)

In [1]:
"""
pwcnet_finetune.ipynb

PWC-Net model finetuning.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy
from dataset_mpisintel import MPISintelDataset
from dataset_base import _DEFAULT_DS_TUNE_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_kitti import KITTIDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_FINETUNE_OPTIONS

In [2]:
!nvidia-smi

Mon May 20 14:16:25 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   38C    P0    50W / 250W |    225MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:06:00.0 Off |                  Off |
| N/A   37C    P0    49W / 250W |    223MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## TODO: Set this first!

In [19]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/Vol1/dbstore/datasets/KITTI_stereo_2015/data/' #'/Vol1/dbstore/datasets/sintel/'
    _DATASET_ROOT2 = '/Vol1/dbstore/datasets/sintel/'
    _MPISINTEL_ROOT = _DATASET_ROOT2 + 'Sintel_color'

    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
# gpu_devices = ['/device:GPU:0', '/device:GPU:1']
# controller = '/device:CPU:0'
gpu_devices = ['/device:GPU:0' ,'/device:GPU:1', '/device:GPU:2']
controller = '/device:GPU:1'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 2

# Finetune on `sintel` mix

## Load the dataset

In [21]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TUNE_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'basic'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
#ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'noc'
#ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [22]:
ds_opts['crop_preproc'] = (320,896)

In [23]:
# ds = MPISintelDataset(mode='train_with_val', ds_root=_MPISINTEL_ROOT, options=ds_opts)

In [24]:
ds = KITTIDataset(mode = 'train_with_val', ds_root=_DATASET_ROOT, options=ds_opts)

In [25]:
ds_opts['type'] = 'final'
ds2 = MPISintelDataset(mode='train_with_val', ds_root=_MPISINTEL_ROOT, options=ds_opts)

In [26]:
ds_full = MixedDataset(mode='train_with_val', datasets=[ds, ds2], options=ds_opts)

In [28]:
# Display dataset configuration
ds_full.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (320, 896)
  scale_preproc        None
  type                 final
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             basic
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           6
  mode                 train_with_val
  train size           1203
  val size             38


In [29]:
ckpt_path = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

## Configure the finetuning

In [30]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_FINETUNE_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/'
nn_opts['batch_size'] = 8
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-small model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# Robust loss as described doesn't work, so try the following:
nn_opts['loss_fn'] = 'loss_multiscale' # 'loss_multiscale' # 'loss_robust' # 'loss_robust'
nn_opts['q'] = 1. # 0.4 # 1. # 0.4 # 1.
nn_opts['epsilon'] = 0. # 0.01 # 0. # 0.01 # 0.

In [31]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['init_lr'] = 1e-05
nn_opts['lr_boundaries'] = [80000, 120000, 160000, 200000]
nn_opts['lr_values'] = [1e-05, 5e-06, 2.5e-06, 1.25e-06, 6.25e-07]
nn_opts['max_steps'] = 200000
nn_opts['sparse_gt_flow'] = True
# Below,we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / 8)
nn_opts['cyclic_lr_stepsize'] = int(nn_opts['cyclic_lr_stepsize'] * 8 / 8)

In [32]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model towers...
  Building tower_0...
  ...tower_0 built.
  Building tower_1...
  ...tower_1 built.
  Building tower_2...
  ...tower_2 built.
... model towers built.
Initializing from pre-trained model at ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000 for finetuning...

INFO:tensorflow:Restoring parameters from ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
... model initialized

Model Configuration:
  verbose                True
  ckpt_path              ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
  ckpt_dir               ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 320, 896, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [320, 896, 2]
  train_mode             fine-tune
  adapt_info             None
  sparse_gt_flow         True
  display_step           100
  snapshot_step          1000

## Finetune the model

In [ ]:
# Train the model
nn.train()

Start finetuning...
2019-05-20 16:18:41 Iter 100 [Train]: loss=242.27, epe=7.77, lr=0.000010, samples/sec=13.2, sec/step=1.819, eta=4 days, 5:00:20
2019-05-20 16:21:10 Iter 200 [Train]: loss=180.80, epe=5.78, lr=0.000010, samples/sec=17.2, sec/step=1.391, eta=3 days, 5:13:32
2019-05-20 16:23:40 Iter 300 [Train]: loss=169.97, epe=5.41, lr=0.000010, samples/sec=17.1, sec/step=1.403, eta=3 days, 5:49:48
2019-05-20 16:26:10 Iter 400 [Train]: loss=162.70, epe=5.17, lr=0.000010, samples/sec=17.2, sec/step=1.397, eta=3 days, 5:28:02
2019-05-20 16:28:41 Iter 500 [Train]: loss=158.94, epe=5.03, lr=0.000010, samples/sec=16.9, sec/step=1.417, eta=3 days, 6:30:24
2019-05-20 16:31:11 Iter 600 [Train]: loss=156.78, epe=4.96, lr=0.000010, samples/sec=17.2, sec/step=1.399, eta=3 days, 5:30:29
2019-05-20 16:33:40 Iter 700 [Train]: loss=155.79, epe=4.93, lr=0.000010, samples/sec=17.2, sec/step=1.392, eta=3 days, 5:04:56
2019-05-20 16:36:10 Iter 800 [Train]: loss=153.57, epe=4.85, lr=0.000010, samples/se

2019-05-20 18:32:13 Iter 5400 [Train]: loss=140.32, epe=4.44, lr=0.000010, samples/sec=17.4, sec/step=1.382, eta=3 days, 2:41:33
2019-05-20 18:34:43 Iter 5500 [Train]: loss=139.50, epe=4.42, lr=0.000010, samples/sec=17.1, sec/step=1.405, eta=3 days, 3:53:25
2019-05-20 18:37:12 Iter 5600 [Train]: loss=139.44, epe=4.42, lr=0.000010, samples/sec=17.2, sec/step=1.394, eta=3 days, 3:18:00
2019-05-20 18:39:42 Iter 5700 [Train]: loss=138.41, epe=4.39, lr=0.000010, samples/sec=17.1, sec/step=1.403, eta=3 days, 3:42:35
2019-05-20 18:42:12 Iter 5800 [Train]: loss=138.47, epe=4.38, lr=0.000010, samples/sec=17.1, sec/step=1.405, eta=3 days, 3:47:55
2019-05-20 18:44:43 Iter 5900 [Train]: loss=139.15, epe=4.40, lr=0.000010, samples/sec=17.0, sec/step=1.412, eta=3 days, 4:06:48
2019-05-20 18:47:14 Iter 6000 [Train]: loss=139.13, epe=4.41, lr=0.000010, samples/sec=17.0, sec/step=1.414, eta=3 days, 4:12:15
2019-05-20 18:47:14 Iter 6000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-l

2019-05-20 20:45:37 Iter 10700 [Train]: loss=134.26, epe=4.24, lr=0.000010, samples/sec=17.2, sec/step=1.398, eta=3 days, 1:29:51
2019-05-20 20:48:07 Iter 10800 [Train]: loss=134.47, epe=4.25, lr=0.000010, samples/sec=17.2, sec/step=1.398, eta=3 days, 1:28:53
2019-05-20 20:50:36 Iter 10900 [Train]: loss=134.51, epe=4.26, lr=0.000010, samples/sec=17.2, sec/step=1.397, eta=3 days, 1:23:52
2019-05-20 20:53:05 Iter 11000 [Train]: loss=133.82, epe=4.24, lr=0.000010, samples/sec=17.3, sec/step=1.389, eta=3 days, 0:54:57
2019-05-20 20:53:05 Iter 11000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-11000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-11000
2019-05-20 20:55:46 Iter 11100 [Train]: loss=134.69, epe=4.26, lr=0.000010, samples/sec=17.0, sec/step=1.409, eta=3 days, 1:56:19
2019-05-20 20:58:15 Iter 11200 [Train]: loss=134.23, epe=4.25, 

2019-05-20 22:38:10 Iter 15900 [Train]: loss=133.29, epe=4.22, lr=0.000010, samples/sec=20.4, sec/step=1.176, eta=2 days, 12:07:09
2019-05-20 22:40:14 Iter 16000 [Train]: loss=132.42, epe=4.20, lr=0.000010, samples/sec=21.0, sec/step=1.142, eta=2 days, 10:22:57
2019-05-20 22:40:14 Iter 16000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-16000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-16000
2019-05-20 22:42:32 Iter 16100 [Train]: loss=132.53, epe=4.20, lr=0.000010, samples/sec=20.4, sec/step=1.177, eta=2 days, 12:06:02
2019-05-20 22:44:36 Iter 16200 [Train]: loss=132.38, epe=4.20, lr=0.000010, samples/sec=20.9, sec/step=1.147, eta=2 days, 10:35:09
2019-05-20 22:46:44 Iter 16300 [Train]: loss=132.03, epe=4.19, lr=0.000010, samples/sec=20.3, sec/step=1.183, eta=2 days, 12:21:50
2019-05-20 22:48:51 Iter 16400 [Train]: loss=133.00, epe=4

Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-21000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-21000
2019-05-21 00:28:05 Iter 21100 [Train]: loss=130.72, epe=4.15, lr=0.000010, samples/sec=20.6, sec/step=1.163, eta=2 days, 9:47:42
2019-05-21 00:30:11 Iter 21200 [Train]: loss=131.43, epe=4.17, lr=0.000010, samples/sec=20.5, sec/step=1.169, eta=2 days, 10:03:07
2019-05-21 00:32:19 Iter 21300 [Train]: loss=131.06, epe=4.16, lr=0.000010, samples/sec=20.5, sec/step=1.170, eta=2 days, 10:05:32
2019-05-21 00:34:25 Iter 21400 [Train]: loss=130.54, epe=4.15, lr=0.000010, samples/sec=20.6, sec/step=1.164, eta=2 days, 9:44:14
2019-05-21 00:36:31 Iter 21500 [Train]: loss=131.70, epe=4.18, lr=0.000010, samples/sec=20.7, sec/step=1.160, eta=2 days, 9:32:06
2019-05-21 00:38:37 Iter 21600 [Train]: loss=131.40, epe=4.17, lr=0.000010, samples/sec=20.6, sec/step=1.164, eta=2 d

...

2019-05-21 07:46:31 Iter 41800 [Train]: loss=129.09, epe=4.12, lr=0.000010, samples/sec=20.7, sec/step=1.159, eta=2 days, 2:56:36
2019-05-21 07:48:38 Iter 41900 [Train]: loss=129.83, epe=4.14, lr=0.000010, samples/sec=20.6, sec/step=1.163, eta=2 days, 3:05:24
2019-05-21 07:50:45 Iter 42000 [Train]: loss=128.97, epe=4.11, lr=0.000010, samples/sec=20.5, sec/step=1.173, eta=2 days, 3:29:36
2019-05-21 07:50:45 Iter 42000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-42000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-42000
2019-05-21 07:53:08 Iter 42100 [Train]: loss=129.44, epe=4.13, lr=0.000010, samples/sec=20.2, sec/step=1.187, eta=2 days, 4:03:55
2019-05-21 07:55:18 Iter 42200 [Train]: loss=129.07, epe=4.11, lr=0.000010, samples/sec=20.4, sec/step=1.175, eta=2 days, 3:30:16
2019-05-21 07:57:23 Iter 42300 [Train]: loss=129.42, epe=4.13, 

2019-05-21 09:37:31 Iter 47000 [Train]: loss=128.73, epe=4.11, lr=0.000010, samples/sec=20.7, sec/step=1.158, eta=2 days, 1:12:40
2019-05-21 09:37:31 Iter 47000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-47000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-47000
2019-05-21 09:39:53 Iter 47100 [Train]: loss=128.66, epe=4.11, lr=0.000010, samples/sec=20.5, sec/step=1.173, eta=2 days, 1:47:58
2019-05-21 09:41:59 Iter 47200 [Train]: loss=128.63, epe=4.11, lr=0.000010, samples/sec=20.6, sec/step=1.163, eta=2 days, 1:20:48
2019-05-21 09:44:03 Iter 47300 [Train]: loss=128.52, epe=4.10, lr=0.000010, samples/sec=20.9, sec/step=1.146, eta=2 days, 0:37:47
2019-05-21 09:46:10 Iter 47400 [Train]: loss=128.34, epe=4.10, lr=0.000010, samples/sec=20.5, sec/step=1.170, eta=2 days, 1:35:09
2019-05-21 09:48:16 Iter 47500 [Train]: loss=128.66, epe=4.11, 

... model saved in ./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-52000
2019-05-21 11:36:06 Iter 52100 [Train]: loss=128.17, epe=4.10, lr=0.000010, samples/sec=17.0, sec/step=1.411, eta=2 days, 9:59:13
2019-05-21 11:38:36 Iter 52200 [Train]: loss=127.67, epe=4.08, lr=0.000010, samples/sec=17.1, sec/step=1.404, eta=2 days, 9:38:08
2019-05-21 11:41:09 Iter 52300 [Train]: loss=128.95, epe=4.12, lr=0.000010, samples/sec=16.7, sec/step=1.436, eta=2 days, 10:55:31
2019-05-21 11:43:39 Iter 52400 [Train]: loss=128.19, epe=4.09, lr=0.000010, samples/sec=17.2, sec/step=1.399, eta=2 days, 9:20:25
2019-05-21 11:46:09 Iter 52500 [Train]: loss=128.10, epe=4.09, lr=0.000010, samples/sec=17.0, sec/step=1.411, eta=2 days, 9:49:55
2019-05-21 11:48:41 Iter 52600 [Train]: loss=127.97, epe=4.08, lr=0.000010, samples/sec=17.0, sec/step=1.413, eta=2 days, 9:51:12
2019-05-21 11:51:11 Iter 52700 [Train]: loss=128.27, epe=4.10, lr=0.000010, samples/sec=17.0, sec/step=1.410, eta=2 days, 9:41:32
2019-05-2

2019-05-21 13:48:08 Iter 57300 [Train]: loss=127.54, epe=4.08, lr=0.000010, samples/sec=17.1, sec/step=1.404, eta=2 days, 7:39:10
2019-05-21 13:50:42 Iter 57400 [Train]: loss=127.54, epe=4.07, lr=0.000010, samples/sec=16.6, sec/step=1.446, eta=2 days, 9:16:44
2019-05-21 13:53:13 Iter 57500 [Train]: loss=127.24, epe=4.07, lr=0.000010, samples/sec=17.0, sec/step=1.409, eta=2 days, 7:46:21
2019-05-21 13:55:43 Iter 57600 [Train]: loss=127.82, epe=4.09, lr=0.000010, samples/sec=17.2, sec/step=1.399, eta=2 days, 7:20:53
2019-05-21 13:58:15 Iter 57700 [Train]: loss=128.53, epe=4.11, lr=0.000010, samples/sec=16.8, sec/step=1.427, eta=2 days, 8:24:08
2019-05-21 14:00:46 Iter 57800 [Train]: loss=127.76, epe=4.08, lr=0.000010, samples/sec=17.0, sec/step=1.413, eta=2 days, 7:49:50
2019-05-21 14:03:19 Iter 57900 [Train]: loss=128.40, epe=4.11, lr=0.000010, samples/sec=16.7, sec/step=1.435, eta=2 days, 8:38:23
2019-05-21 14:05:49 Iter 58000 [Train]: loss=127.35, epe=4.08, lr=0.000010, samples/sec=17

2019-05-21 16:03:13 Iter 62500 [Train]: loss=127.69, epe=4.09, lr=0.000010, samples/sec=16.4, sec/step=1.462, eta=2 days, 7:51:07
2019-05-21 16:05:51 Iter 62600 [Train]: loss=127.68, epe=4.09, lr=0.000010, samples/sec=16.4, sec/step=1.466, eta=2 days, 7:57:59
2019-05-21 16:08:26 Iter 62700 [Train]: loss=128.09, epe=4.10, lr=0.000010, samples/sec=16.7, sec/step=1.440, eta=2 days, 6:55:16
2019-05-21 16:11:03 Iter 62800 [Train]: loss=127.45, epe=4.08, lr=0.000010, samples/sec=16.6, sec/step=1.446, eta=2 days, 7:07:08
2019-05-21 16:13:38 Iter 62900 [Train]: loss=126.95, epe=4.06, lr=0.000010, samples/sec=16.7, sec/step=1.438, eta=2 days, 6:45:36
2019-05-21 16:16:15 Iter 63000 [Train]: loss=128.88, epe=4.12, lr=0.000010, samples/sec=16.6, sec/step=1.450, eta=2 days, 7:10:46
2019-05-21 16:16:15 Iter 63000 [Val]: loss=nan, epe=nan
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-cyclic-KITTI2015_finetuned/pwcnet.ckpt-63000 is not in all_model_checkpoint_paths. Manually adding it.
... model sav

## Training log

Here are the training curves for the run above:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/loss.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/epe.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/lr.png)

Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val1.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val2.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val3.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val4.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val5.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val6.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val7.png)
![](img/pwcnet-sm-6-2-cyclic-chairsthingsmix_finetuned/val8.png)